In [1]:
from pyspark import SparkContext, SparkConf
from operator import add

conf = SparkConf()

conf.setMaster("spark://192.168.2.173:7077") \
        .setAppName("tryA")\
        .set("spark.dynamicAllocation.enabled", True)\
        .set("spark.shuffle.service.enabled", True)\
        .set("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .set("spark.executor.cores",2)\
        .set("spark.driver.port",9998)\
        .set("spark.blockManager.port",10005)


context = SparkContext(conf=conf)
context.getOrCreate()

<SparkContext master=spark://192.168.2.173:7077 appName=tryA>

In [2]:
pointer_file = context.textFile("hdfs://192.168.2.173:9000/user/ubuntu/TN_STAID000001.txt")

In [4]:
pointer_file.take(20)

['EUROPEAN CLIMATE ASSESSMENT & DATASET (ECA&D), file created on: 20-02-2021',
 'THESE DATA CAN BE USED FOR NON-COMMERCIAL RESEARCH AND EDUCATION PROVIDED THAT THE FOLLOWING SOURCE IS ACKNOWLEDGED: ',
 '',
 'Klein Tank, A.M.G. and Coauthors, 2002. Daily dataset of 20th-century surface',
 'air temperature and precipitation series for the European Climate Assessment.',
 'Int. J. of Climatol., 22, 1441-1453.',
 'Data and metadata available at http://www.ecad.eu',
 '',
 'FILE FORMAT (MISSING VALUE CODE = -9999):',
 '',
 '01-06 STAID: Station identifier',
 '08-13 SOUID: Source identifier',
 '15-22 DATE : Date YYYYMMDD',
 '24-28 TN   : Minimum temperature in 0.1 &#176;C',
 "30-34 Q_TN : quality code for TN (0='valid'; 1='suspect'; 9='missing')",
 '',
 'This is the blended series of station VAEXJOE, SWEDEN (STAID: 1)',
 'Blended and updated with sources:1 35383 ',
 'See files sources.txt and stations.txt for more info.',
 '']

In [5]:
def add(a, b):
    # commutative and associative!
    return a + b

rdd = context.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 3)

result = rdd\
            .filter(lambda x: x % 2 == 0)\
            .map(lambda x: x ** 2)\
            .reduce(add)

print(result)

220


In [2]:
context.stop()